# 线性回归的简洁实现
本节将使⽤MXNet提供的Gluon接口更⽅便地实现线性回归的训练。
## 生成数据集

In [1]:
from mxnet import nd, autograd
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(shape=(num_examples, num_inputs))
labels = true_w[0]*features[:,0] + true_w[1]*features[:,1] + true_b
labels += nd.random.normal(scale=0.01,shape=labels.shape)

In [2]:
labels.take(nd.array(range(5)))


[ 4.879625   4.2968144 -3.4331114 11.099875  -3.8235688]
<NDArray 5 @cpu(0)>

## 读取数据
Gluon提供了data包来读取数据。由于data常⽤作变量名，我们将导⼊的data模块⽤添加
了Gluon⾸字⺟的假名gdata代替。在每⼀次迭代中，我们将随机读取包含10个数据样本的小
批量。

In [3]:
from mxnet.gluon import data as gdata
batch_size = 10
# 将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features, labels)
# 随机读取⼩批量
data_iter = gdata.DataLoader(dataset, batch_size, True)

In [4]:
for X, y in data_iter:
    print(X, y)
    break;
    


[[-0.4070257  -0.760578  ]
 [ 1.591328    0.24141619]
 [-0.28439447  1.0559008 ]
 [ 0.50472885 -0.48745614]
 [-0.43371913  1.3150975 ]
 [ 0.22842942  0.62486184]
 [-0.5400787   0.4005976 ]
 [-0.53560084 -0.00894159]
 [-0.4658121  -0.28205368]
 [-0.04197302 -0.08312167]]
<NDArray 10x2 @cpu(0)> 
[ 5.9727497   6.545585    0.05495567  6.874393   -1.1574751   2.5135262
  1.7483729   3.1680815   4.217416    4.405445  ]
<NDArray 10 @cpu(0)>


## 定义模型
Gluon提供了⼤量预定义的层，这使我们只需关注使⽤哪些层来构造模型。下⾯将介绍如何使⽤Gluon更简洁地定义线性回归。
⾸先，导⼊nn模块。实际上，“nn”是neural networks（神经⽹络）的缩写。顾名思义，该模块定
义了⼤量神经⽹络的层。我们先定义⼀个模型变量net，它是⼀个Sequential实例。在Gluon中，
Sequential实例可以看作是⼀个串联各个层的容器。在构造模型时，我们在该容器中依次添加
层。当给定输⼊数据时，容器中的每⼀层将依次计算并将输出作为下⼀层的输⼊。

In [5]:
from mxnet.gluon import nn
net = nn.Sequential()

回顾线性回归在神经⽹络图中的表⽰。作为⼀个单层神经⽹络，线性回归输出层中的神
经元和输⼊层中各个输⼊完全连接。因此，线性回归的输出层⼜叫全连接层。
在Gluon中，全连接层是⼀个Dense实例。我们定义该层输出个数为1。

In [6]:
net.add(nn.Dense(1))

值得⼀提的是，在Gluon中我们⽆须指定每⼀层输⼊的形状，例如线性回归的输⼊个数。当模型
得到数据时，例如后⾯执⾏net(X)时，模型将⾃动推断出每⼀层的输⼊个数。我们将在之后“深
度学习计算”⼀章详细介绍这种机制。 Gluon的这⼀设计为模型开发带来便利
## 初始化模型参数
在使⽤net前，我们需要初始化模型参数，如线性回归模型中的权重和偏差。我们从MXNet导
⼊init模块。该模块提供了模型参数初始化的各种⽅法。这⾥的init是initializer的缩写
形式。我们通过init.Normal(sigma=0.01)指定权重参数每个元素将在初始化时随机采样于
均值为0、标准差为0.01的正态分布。偏差参数默认会初始化为零。

In [7]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

## 定义损失函数
在Gluon中， loss模块定义了各种损失函数。我们⽤假名gloss代替导⼊的loss模块，并直接
使⽤它提供的平⽅损失作为模型的损失函数。

In [8]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss() # 平⽅损失⼜称L2范数损失

## 定义优化算法
在导⼊Gluon后，我们创建⼀个Trainer实例，并指定学习率为0.03的小批量随机梯度下降（sgd）为优化算法。
该优化算法将⽤来迭代net实例所有通过add函数嵌套的层所包含的全部参数。这些参数可以通过collect_params函数获取

In [9]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

## 训练模型
在使⽤Gluon训练模型时，我们通过调⽤Trainer实例的step函数来迭代模型参数。我
们提到，由于变量l是⻓度为batch_size的⼀维NDArray，执⾏l.backward()等价于执⾏l.
sum().backward()。按照小批量随机梯度下降的定义，我们在step函数中指明批量⼤小，从
而对批量中样本梯度求平均

In [13]:
num_epoch = 3
for epoch in range(1, num_epoch+1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    train_l = loss(net(features), labels)
    print('epoch %d, loss %f' % (epoch, train_l.mean().asnumpy()))

epoch 1, loss 0.000048
epoch 2, loss 0.000048
epoch 3, loss 0.000048


下⾯我们分别⽐较学到的模型参数和真实的模型参数。我们从net获得需要的层，并访问其权重
（weight）和偏差（bias）。学到的参数和真实的参数很接近。

In [14]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 2.000011 -3.399709]]
 <NDArray 1x2 @cpu(0)>)

In [15]:
true_b, dense.bias.data()

(4.2, 
 [4.200454]
 <NDArray 1 @cpu(0)>)

使⽤Gluon可以更简洁地实现模型。
- 在Gluon中， data模块提供了有关数据处理的⼯具， nn模块定义了⼤量神经⽹络的层，
- loss模块定义了各种损失函数。
- MXNet的initializer模块提供了模型参数初始化的各种⽅法。